In [ ]:
import pandas as pd
import requests as re
from pathlib import Path



This code extracts raw unprepared statistics for all matches from given competition and saves it into csv-file. You need Cyanide API key for this. To get the key contact Cyanide Studio Team com@cyanide-studio.com. I keep my private key in txt-file (it looks like https://web.cyanide-studio.com/ws/?bb=3&key=MYPRIVATEKEY)

In [ ]:
with open('api_key.txt', 'r') as file:
    lines = file.readlines()
api_key = str(lines[0]).split('=')[-1]

For extracting data you need competition_id (competition_name may be not unique). If you want to get matches from official ladder (like I do), you may execute following code:
```python
url = f'https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}'

response = re.get(url)

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        for row in df[['id', 'name']].itertuples():
            print(f"{row.Index}: id={row.id}, name={row.name}")
    else:
        print("No competitions in this league.")
else:
    print(f'Error: {response.status_code}')
```
and choose id.

If you want to get matches from another league, you need its ID or name. Try something like this:
```python
league_name = 'Russian Blood Bowl League'
url = f"https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}&league_name={league_name}"

response = re.get(url)

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        for row in df[['id', 'name']].itertuples():
            print(f"{row.Index}: id={row.id}, name={row.name}")
    else:
        print("No competitions in this league.")
else:
    print(f'Error: {response.status_code}')
```

Besides competition_id you have to define several parameters: 
- league_name: league your competition is from (default is Official League);
- platform: doesn't matter much, used for naming saved csv files. I use "pc", "playstation", "xbox" or "crossplay";     
- chunk_limit: defines amount of rows you'll get for 1 iteration, 1000 is max;  
- request_limit: defines max amount of requests you code will make, current limits are 1000 per hour and 10000 per day, you have not to exceed these limits.

In [ ]:
competition_id = '8e2ee0c8-3275-11ef-9e7e-bc24112ec32e'
league_name = 'Russian Blood Bowl League'
platform = 'pc'
chunk_limit = 1000
request_limit = 200

In [ ]:
url = f"https://web.cyanide-studio.com/ws/bb3/competitions/?key={api_key}&league_name={league_name}&limit=1000"
response = re.get(url)
break_flag = 0 # If we need to break while cycle
continue_flag = 0 # If we restart with previously saved data

if response.status_code == 200:
    print('Response: OK')
    data = response.json()
    
    if 'competitions' in data:
        df = pd.json_normalize(data['competitions'])
        
        if competition_id in df.id.values:
            competition_start = df[df.id == competition_id].date_created.values[0]
            competition_name = df[df.id == competition_id].name.values[0].replace(' ', '_')

            if Path('temporary.csv').exists():
                df = pd.read_csv('temporary.csv')
                n = 0
                print('Continue from previously loaded data.')
                continue_flag = 1
    
            else:
                print('Start extracting data.')
                url = f'''https://web.cyanide-studio.com/ws/bb3/matches/?key={api_key}\
&competition_id={competition_id}\
&start={competition_start}\
&limit={chunk_limit}'''
                n = 1
                response = re.get(url)

                if response.status_code == 200:
                    print(f'{n} request: OK')
                    data = response.json()
                    df = pd.json_normalize(data['matches'])

                else:
                    print(f'{n} request: ERROR: {response.status_code}')
                    df = pd.DataFrame()

            try:
                anchor_date = df.started.min()
                combined_df = df.copy()

            except Exception as e:
                print(f"Something broke: {e}. May be, no matches in this competition.")
                break_flag = 1

            try:
                while (len(df) == chunk_limit and n < request_limit) or continue_flag:
                    
                    if len((df[df.started != anchor_date])) == 0 and not continue_flag:
                        print(f"Our chunks are too small. Or too many matches start at the same second. Can't proceed.")
                        print(f"Make chunks larger, if possible.")
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.to_csv(f'temporary.csv', index=False)
                        print(f"Partially extracted data is saved to 'temporary.csv', you can continue later from this point.")
                        break_flag = 1
                        break

                    continue_flag = 0
                    n += 1
                    url = f'''https://web.cyanide-studio.com/ws/bb3/matches/?key={api_key}\
&competition_id={competition_id}\
&start={competition_start}\
&end={anchor_date}
&limit={chunk_limit}'''
                    response = re.get(url)
                    
                    if response.status_code == 200:
                        print(f'{n} request: OK')
                        
                    else:
                        print(f'{n} request: ERROR: {response.status_code}')
                    
                    data = response.json()
                    df = pd.json_normalize(data['matches'])
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
                    anchor_date = combined_df.started.min()

                if not break_flag:
                    print(f"Data extracted with {n} requests to 'matches' endpoint.")
                
                    if n >= request_limit:
                        print('Request limit exceeded. Try to continue later.')
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.to_csv(f'temporary.csv', index=False)
                        print(f"Partially extracted data is saved to 'temporary.csv'")
                
                    else:
                        combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                        combined_df.drop('api_match', axis=1, inplace=True)
                        combined_df.info()
                        combined_df.to_csv(f'{competition_name}_{platform}_raw_data.csv', index=False)
                        print(f"Data is saved to '{competition_name}_{platform}_raw_data.csv'")
                        
                        if Path('temporary.csv').exists():
                            Path('temporary.csv').unlink()
                            print(f'temporary.csv deleted.')
                
            except Exception as e:
                print(f"Something broke: {e}. Data partially extracted with {n} requests to 'matches' endpoint.")
                combined_df.drop_duplicates(subset=['uuid'], inplace=True)
                combined_df.to_csv(f'temporary.csv', index=False)
                print(f"Partially extracted data is saved to 'temporary.csv', you can continue later from this point.")
        
        else:
            print("No competition with this ID in this league.")
            
    else:
        print("No competitions in this league.")
        
else:
    print(f'Error: {response.status_code}')